In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

#code pour recup les raw data

# LOAD DES DONNEES

# Fonction pour charger les données à partir d'un dossier
def load_data(folder):
    data = []
    
    # Parcourez tous les fichiers CSV dans le dossier principal
    for label, file in enumerate(os.listdir(folder)):
        file_path = os.path.join(folder, file)
        
        # Vérifiez si l'élément est un fichier CSV
        if file.endswith(".csv") and os.path.isfile(file_path):
            df = pd.read_csv(file_path)
            
            # Aplatir les données et les ajouter à la liste
            data.append(df.values.flatten())  
    for i in range(len(data)) :    # Séparer les éléments des sous-listes en sous-listes pour que test_data[i][j][k] corresponde au ième fichier, jème ligne et un sélectionne x, y, polarity ou time avec k. 
        data[i] = np.array(data[i]).reshape(-1, 4).tolist()
    return data


In [ ]:
def create_data_dictionary(folder_path):
    dict_data = {}

    for i, subfolder in enumerate(os.listdir(folder_path)):
        subfolder_path = os.path.join(folder_path, subfolder)
        data_list = load_data(subfolder_path)
        dict_data[str(i)] = data_list

    return dict_data

# Example usage
folder_path = '/kaggle/input/smemi309-final-evaluation-challenge-2023/train10/train10'
data_dict = create_data_dictionary(folder_path)

# Accessing values in the data_dict
for label, data_list in data_dict.items():
    print(f"Label: {label}, Data List Length: {len(data_list)}")
    

In [ ]:
# Example usage
folder_path = '/kaggle/input/smemi309-final-evaluation-challenge-2023/test10'
data_dict_test = create_data_dictionary(folder_path)

# Accessing values in the data_dict
for label, data_list in data_dict_test.items():
    print(f"Label: {label}, Data List Length: {len(data_list)}")

Preprocessing des données

In [ ]:
def interv(data,n): # Sépare le dataframe en n intervalle de temps
    Temps = []
    L = len(data)
    for i in range(n):
        Temps.append(i * len(data) // n)
    Temps.append(L-1)
    return Temps

def separedata(data,n):
    D = []
    S = interv(data,n)
    for i in range(1,len(S)):
        D.append(data[S[i-1]:S[i]])
    return D

Mise sous form matricielle des données

In [ ]:
def matr(data):   #n = nombre d'intervalle    (à faire avec data = test_data[0])
    list_data = []
    for j in range(len(data)):
        list_data.append([data[j][0], data[j][1], data[j][2]])      #[[1,2,3],[4,5,6],[7,8,9]]
    M = np.zeros([224,90])
    for i in range(len(data)):
            if list_data[i][2] == 1 :
                M[list_data[i][0],list_data[i][1]] += 1   #M[x,y] = somme des polarités
            else :
                M[list_data[i][0],list_data[i][1]] -= 1
    return M

Convolution des matrices

In [ ]:
def sum_by_block(M,Lx,Ly):

    # Calculer les dimensions de la nouvelle matrice B
    nb_lignes_B = M.shape[0] // Lx
    nb_colonnes_B = M.shape[1] // Ly

    # Initialiser la nouvelle matrice B avec des zéros
    B = np.zeros((nb_lignes_B, nb_colonnes_B), dtype=M.dtype)

    # Parcourir les blocs de la matrice M
    for i in range(nb_lignes_B):
        for j in range(nb_colonnes_B):
            # Extraire le bloc correspondant de la matrice M avec les longueurs Lx et Ly
            bloc_M = M[i * Lx:(i + 1) * Lx, j * Ly:(j + 1) * Ly]

            # Ajouter la somme des éléments du bloc à la matrice B
            B[i, j] = np.sum(bloc_M)

    # Sors la matrice résultante B
    return B



In [ ]:
def label_bloc(data, n, Lx, Ly) :
    M = []
    for i in range(32) :
        M.append([])
        for j in range(n) :
            M[i].append(sum_by_block(matr(separedata(data[i],n)[j]), Lx, Ly))
    return M

In [ ]:
def label_bloc_test(data, n, Lx, Ly) :
    M = []
    for i in range(100) :
        M.append([])
        for j in range(n) :
            M[i].append(sum_by_block(matr(separedata(data[i],n)[j]), Lx, Ly))
    return M

In [ ]:
n = 20 # Nombre d'intervalles
Lx = 28 # Longueur bloc
Ly = 15 # Largeur bloc

dict_label_bloc = {}

for key in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
    dict_label_bloc[key] = label_bloc(data_dict[key], n, Lx, Ly)


In [ ]:
n = 20 # Nombre d'intervalles
Lx = 28 # Longueur bloc
Ly = 15 # Largeur bloc

dict_label_bloc_test = {}
dict_label_bloc_test['0'] = label_bloc_test(data_dict_test['0'], n, Lx, Ly)


POUR LE TRAIN


In [ ]:
data_flat = [element for sous_liste in dict_label_bloc.values() for element in sous_liste]
print(np.shape(data_flat))
data_flat=np.array(data_flat)
data_flat2=data_flat.reshape(320,-1)
print(np.shape(data_flat2))

POUR LE TEST


In [ ]:
data_flat_test = [element for sous_liste in dict_label_bloc_test.values() for element in sous_liste]
print(np.shape(data_flat_test))
data_flat_test=np.array(data_flat_test)
data_flat2_test=data_flat_test.reshape(100,-1)
print(np.shape(data_flat2_test))

In [ ]:
#RANDOM FOREST CLASSFIER V1
#Tester pour diffrentes valeurs de n, Lx,Ly et n_estimator

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Chargez les données du dictionnaire
def randomforest(dictionnaire,n_est):
    data_flat = [element for sous_liste in dictionnaire.values() for element in sous_liste]
    data_flat=np.array(data_flat)
    data_flat2=data_flat.reshape(320,-1)

    lab=np.array([[0]*32,[1]*32,[2]*32,[3]*32,[4]*32,[5]*32,[6]*32,[7]*32,[8]*32,[9]*32])
    lab = [element for sous_liste in lab for element in sous_liste]

    for i in range(len(lab)):
        lab[i]=np.array(lab[i])
    # Séparez les données en ensemble de test et d'apprentissage

    X_train, X_test, y_train, y_test = train_test_split(data_flat2, lab, test_size=0.4, random_state=7)

    # Créez un modèle Random Forest fair varier le n__estimator
    clf = RandomForestClassifier(n_estimators=n_est)

    # Entraînez le modèle sur l'ensemble d'apprentissage
    clf.fit(X_train, y_train)

    # Prédisez les sorties pour l'ensemble de test
    y_pred = clf.predict(X_test)

    # Calculez la précision du modèle
    print("Précision :", accuracy_score(y_test, y_pred))
    return accuracy_score(y_test, y_pred)

In [ ]:
#RANDOM FOREST CLASSFIER
#Tester pour diffrentes valeurs de n, Lx,Ly et n_estimator

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def RandomForest(dictionnaire, Nb_estim, Random):
    # Chargez les données du dictionnaire
    data_flat = [element for sous_liste in dictionnaire.values() for element in sous_liste]
    data_flat=np.array(data_flat)
    data_flat2=data_flat.reshape(320,-1)

    lab=np.array([[0]*32,[1]*32,[2]*32,[3]*32,[4]*32,[5]*32,[6]*32,[7]*32,[8]*32,[9]*32])
    lab = [element for sous_liste in lab for element in sous_liste]

    for i in range(len(lab)):
        lab[i]=np.array(lab[i])
    # Séparez les données en ensemble de test et d'apprentissage

    X_train, X_test, y_train, y_test = train_test_split(data_flat2, lab, test_size=0.25, random_state=Random)

    # Créez un modèle Random Forest fair varier le n__estimator
    clf = RandomForestClassifier(n_estimators=Nb_estim)

    # Entraînez le modèle sur l'ensemble d'apprentissage
    clf.fit(X_train, y_train)

    # Prédisez les sorties pour l'ensemble de test
    y_pred = clf.predict(X_test)

    # Calculez la précision du modèle
    """print("Précision :", accuracy_score(y_test, y_pred))"""
    return accuracy_score(y_test, y_pred)

Gridsearch de parametre optimal

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

def RandomForestGridSearch(dictionnaire, Random_state):
    # Chargez les données du dictionnaire
    data_flat = [element for sous_liste in dictionnaire.values() for element in sous_liste]
    data_flat = np.array(data_flat)
    data_flat2 = data_flat.reshape(320, -1)

    lab = np.array([[0] * 32, [1] * 32, [2] * 32, [3] * 32, [4] * 32, [5] * 32, [6] * 32, [7] * 32, [8] * 32, [9] * 32])
    lab = [element for sous_liste in lab for element in sous_liste]

    for i in range(len(lab)):
        lab[i] = np.array(lab[i])

    # Séparez les données en ensemble de test et d'apprentissage
    X_train, X_test, y_train, y_test = train_test_split(data_flat2, lab, test_size=0.25, random_state=random_state)

    # Paramètres à tester
    param_grid = {'n_estimators': np.arange(600,1000,50)}

    # Créez un modèle Random Forest
    clf = RandomForestClassifier()

    # Créez un objet GridSearchCV
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

    # Entraînez le modèle sur l'ensemble d'apprentissage
    grid_search.fit(X_train, y_train)

    # Meilleurs paramètres
    best_params = grid_search.best_params_

    # Utilisez le meilleur modèle pour prédire les sorties pour l'ensemble de test
    y_pred = grid_search.predict(X_test)

    # Calculez la précision du modèle avec les meilleurs paramètres
    best_accuracy = accuracy_score(y_test, y_pred)

    return best_params, best_accuracy

# Exemple d'utilisation
random_state = 42  # Vous pouvez choisir n'importe quelle valeur pour la reproductibilité
best_params, best_accuracy = RandomForestGridSearch(dict_label_bloc, 7)

print("Meilleurs paramètres:", best_params)
print("Précision avec meilleurs paramètres:", best_accuracy)

Graphe

In [ ]:
# Graphe RandomForest en fonction du nombre d'estimateurs

import matplotlib.pyplot as plt

N = np.arange(25,1200,25)
RF = []
for i in range(len(N)):
    moy = []
    for j in range(10):
        moy.append(RandomForest(dict_label_bloc, N[i], i)) # random_state = j
    moy = np.mean(moy)
    RF.append(moy)

plt.plot(N, RF, color = 'green', marker = 'o', label = 'Random Forest')
plt.legend()

print('Précision maximum :', np.max(RF), ' atteint pour', N[np.argmax(RF)],' estimateurs')
print('Variance de la précision RandomForest :', np.var(RF))

random forest avec adaboosting

In [ ]:
# Random Forest avec AdaBoost

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def RandomForestAdaBoost(dictionnaire, Nb_estimRF, Nb_estimAB, Random_state):
    # Charger les données du dictionnaire
    data_flat = [element for sous_liste in dictionnaire.values() for element in sous_liste]
    data_flat = np.array(data_flat)
    data_flat2 = data_flat.reshape(320, -1)

    lab = np.array([[0] * 32, [1] * 32, [2] * 32, [3] * 32, [4] * 32, [5] * 32, [6] * 32, [7] * 32, [8] * 32, [9] * 32])
    lab = [element for sous_liste in lab for element in sous_liste]

    for i in range(len(lab)):
        lab[i] = np.array(lab[i])

    # Séparer les données en ensemble de test et d'apprentissage
    X_train, X_test, y_train, y_test = train_test_split(data_flat2, lab, test_size=0.2, random_state=Random_state)

    # Créer un modèle Random Forest en faisant varier le n_estimators
    clf_rf = RandomForestClassifier(n_estimators=Nb_estimRF)
    clf_rf.fit(X_train, y_train)

    # Prédire les sorties pour l'ensemble de test avec Random Forest
    y_pred_rf = clf_rf.predict(X_test)

    # Calculer la précision du modèle Random Forest
    """print("Précision Random Forest:", accuracy_score(y_test, y_pred_rf))"""

    # Créer un modèle AdaBoost
    clf_ab = AdaBoostClassifier(n_estimators=Nb_estimAB, base_estimator=RandomForestClassifier(n_estimators=Nb_estimRF))

    # Entraîner le modèle AdaBoost sur l'ensemble d'apprentissage
    clf_ab.fit(X_train, y_train)

    # Prédire les sorties pour l'ensemble de test avec AdaBoost
    y_pred_ab = clf_ab.predict(X_test)

    # Calculer la précision du modèle AdaBoost
    """print("Précision AdaBoost:", accuracy_score(y_test, y_pred_ab))"""
   
    return accuracy_score(y_test, y_pred_rf), accuracy_score(y_test, y_pred_ab)

Graphe avec adaboosted random forest et sans 

In [ ]:
# Graphe RandomForest avec et sans AdaBoost

import matplotlib.pyplot as plt

N = np.arange(25,1200,25)
RF = []
AB = []
for i in range(len(N)):
    moyRF = []
    moyAB = []
    for j in range(10):
        RFAB = RandomForestAdaBoost(dict_label_bloc, 825, N[i], j) #825 meilleur paramètre pour RF
        moyRF.append(RFAB[0]) # RandomForest
        moyAB.append(RFAB[1]) # AdaBoost
    moyRF = np.mean(moyRF)
    moyAB = np.mean(moyAB)
    RF.append(moyRF)
    AB.append(moyAB)

plt.plot(N, RF, color = 'green', marker = 'o', label = 'Random Forest')
plt.plot(N, AB, color = 'red', marker = 'o', label = 'AdaBoost')
plt.legend()

print('Précision maximum RandomForest avec AdaBoost :', np.max(AB), ' atteint pour', N[np.argmax(AB)],' estimateurs que prend AdaBoost')

GradientBoosting and Xtreme gradient boosting of randomforest

In [ ]:
# Random Forest avec Extrem Grandient Boosting

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def RandomForestGradientBoosting(dictionnaire, Nb_estimRF, Nb_estimGB):
    # Charger les données du dictionnaire
    data_flat = [element for sous_liste in dictionnaire.values() for element in sous_liste]
    data_flat = np.array(data_flat)
    data_flat2 = data_flat.reshape(320, -1)

    lab = np.array([[0] * 32, [1] * 32, [2] * 32, [3] * 32, [4] * 32, [5] * 32, [6] * 32, [7] * 32, [8] * 32, [9] * 32])
    lab = [element for sous_liste in lab for element in sous_liste]

    for i in range(len(lab)):
        lab[i] = np.array(lab[i])

    # Séparer les données en ensemble de test et d'apprentissage
    X_train, X_test, y_train, y_test = train_test_split(data_flat2, lab, test_size=0.25, random_state=7)

    # Créer un modèle Random Forest en faisant varier le n_estimators
    clf_rf = RandomForestClassifier(n_estimators=Nb_estimRF)
    clf_rf.fit(X_train, y_train)

    # Prédire les sorties pour l'ensemble de test avec Random Forest
    y_pred_rf = clf_rf.predict(X_test)

    # Calculer la précision du modèle Random Forest
    print("Précision Random Forest:", accuracy_score(y_test, y_pred_rf))

    # Créer un modèle Gradient Boosting
    clf_gb = GradientBoostingClassifier(n_estimators=Nb_estimGB, learning_rate=0.1, max_depth=3)

    # Entraîner le modèle sur l'ensemble d'apprentissage
    clf_gb.fit(X_train, y_train)

    # Prédire les sorties pour l'ensemble de test avec Gradient Boosting
    y_pred_gb = clf_gb.predict(X_test)

    # Calculer la précision du modèle Gradient Boosting
    print("Précision Gradient Boosting:", accuracy_score(y_test, y_pred_gb))


In [ ]:
# Random Forest avec Extrem Grandient Boosting

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def RandomForestGradientBoosting(dictionnaire, Nb_estimRF, Nb_estimGB):
    # Charger les données du dictionnaire
    data_flat = [element for sous_liste in dictionnaire.values() for element in sous_liste]
    data_flat = np.array(data_flat)
    data_flat2 = data_flat.reshape(320, -1)

    lab = np.array([[0] * 32, [1] * 32, [2] * 32, [3] * 32, [4] * 32, [5] * 32, [6] * 32, [7] * 32, [8] * 32, [9] * 32])
    lab = [element for sous_liste in lab for element in sous_liste]

    for i in range(len(lab)):
        lab[i] = np.array(lab[i])

    # Séparer les données en ensemble de test et d'apprentissage
    X_train, X_test, y_train, y_test = train_test_split(data_flat2, lab, test_size=0.25, random_state=7)

    # Créer un modèle Random Forest en faisant varier le n_estimators
    clf_rf = RandomForestClassifier(n_estimators=Nb_estimRF)
    clf_rf.fit(X_train, y_train)

    # Prédire les sorties pour l'ensemble de test avec Random Forest
    y_pred_rf = clf_rf.predict(X_test)

    # Calculer la précision du modèle Random Forest
    print("Précision Random Forest:", accuracy_score(y_test, y_pred_rf))

    # Créer un modèle Gradient Boosting
    clf_gb = GradientBoostingClassifier(n_estimators=Nb_estimGB, learning_rate=0.1, max_depth=3)

    # Entraîner le modèle sur l'ensemble d'apprentissage
    clf_gb.fit(X_train, y_train)

    # Prédire les sorties pour l'ensemble de test avec Gradient Boosting
    y_pred_gb = clf_gb.predict(X_test)

    # Calculer la précision du modèle Gradient Boosting
    print("Précision Gradient Boosting:", accuracy_score(y_test, y_pred_gb))


Extra trees and graph

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

def ExtraTrees(dictionnaire, Nb_estim, Random):
    # Chargez les données du dictionnaire
    data_flat = [element for sous_liste in dictionnaire.values() for element in sous_liste]
    data_flat = np.array(data_flat)
    data_flat2 = data_flat.reshape(320, -1)

    lab = np.array([[0] * 32, [1] * 32, [2] * 32, [3] * 32, [4] * 32, [5] * 32, [6] * 32, [7] * 32, [8] * 32, [9] * 32])
    lab = [element for sous_liste in lab for element in sous_liste]

    for i in range(len(lab)):
        lab[i] = np.array(lab[i])

    # Séparez les données en ensemble de test et d'apprentissage
    X_train, X_test, y_train, y_test = train_test_split(data_flat2, lab, test_size=0.25, random_state=Random)

    # Créez un modèle Extra Trees Classifier en faisant varier le n__estimator
    clf = ExtraTreesClassifier(n_estimators=Nb_estim)

    # Entraînez le modèle sur l'ensemble d'apprentissage
    clf.fit(X_train, y_train)

    # Prédisez les sorties pour l'ensemble de test
    y_pred = clf.predict(X_test)

    # Calculez la précision du modèle
    """print("Précision :", accuracy_score(y_test, y_pred))"""
    return accuracy_score(y_test, y_pred)

In [ ]:
# Graphe Extra Trees en fonction du nombre d'estimateurs

import matplotlib.pyplot as plt

N = np.arange(50,1000,50)
ET = []
for i in range(len(N)):
    moy = []
    for j in range(10):
        moy.append(ExtraTrees(dict_label_bloc, N[i], j)) # random_state = j
    moy = np.mean(moy)
    ET.append(moy)

plt.plot(N, ET, color = 'green', marker = 'o', label = 'Extra Trees')
plt.legend()

print('Précision maximum :', np.max(ET), ' atteint pour', N[np.argmax(ET)],' estimateurs')
print('Variance de la précision RandomForest :', np.var(ET))

In [ ]:
randomforest(dict_label_bloc,650)

In [ ]:
# Random Forest avec AdaBoost et bagging

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def Rf_Ab_Bg(dictionnaire, Nb_estimRF, Nb_estimAB,Nb_estimBG):
    # Charger les données du dictionnaire
    data_flat = [element for sous_liste in dictionnaire.values() for element in sous_liste]
    data_flat = np.array(data_flat)
    data_flat2 = data_flat.reshape(320, -1)

    lab = np.array([[0] * 32, [1] * 32, [2] * 32, [3] * 32, [4] * 32, [5] * 32, [6] * 32, [7] * 32, [8] * 32, [9] * 32])
    lab = [element for sous_liste in lab for element in sous_liste]

    for i in range(len(lab)):
        lab[i] = np.array(lab[i])

    # Séparer les données en ensemble de test et d'apprentissage
    X_train, X_test, y_train, y_test = train_test_split(data_flat2, lab, test_size=0.25, random_state=7)

    ###RANDOM FOREST
    # Créer un modèle Random Forest en faisant varier le n_estimators
    clf_rf = RandomForestClassifier(n_estimators=Nb_estimRF)
    clf_rf.fit(X_train, y_train)

    # Prédire les sorties pour l'ensemble de test avec Random Forest
    y_pred_rf = clf_rf.predict(X_test)

    # Calculer la précision du modèle Random Forest
    print("Précision Random Forest:", accuracy_score(y_test, y_pred_rf))
    
    ###ADABOOST
    # Créer un modèle AdaBoost
    clf_ab = AdaBoostClassifier(n_estimators=Nb_estimAB, base_estimator=RandomForestClassifier(n_estimators=Nb_estimRF))

    # Entraîner le modèle AdaBoost sur l'ensemble d'apprentissage
    clf_ab.fit(X_train, y_train)

    # Prédire les sorties pour l'ensemble de test avec AdaBoost
    y_pred_ab = clf_ab.predict(X_test)

    # Calculer la précision du modèle AdaBoost
    print("Précision AdaBoost:", accuracy_score(y_test, y_pred_ab))
    
    ###BAGGING
    ###ADABOOST
    # Créer un modèle Bagging
    clf_bg = BaggingClassifier(n_estimators=Nb_estimBG, base_estimator=RandomForestClassifier(n_estimators=Nb_estimRF))

    # Entraîner le modèle AdaBoost sur l'ensemble d'apprentissage
    clf_bg.fit(X_train, y_train)

    # Prédire les sorties pour l'ensemble de test avec AdaBoost
    y_pred_bg = clf_ab.predict(X_test)

    # Calculer la précision du modèle AdaBoost
    print("Précision Bagging:", accuracy_score(y_test, y_pred_ab))
    
    return(accuracy_score(y_test, y_pred_rf),accuracy_score(y_test, y_pred_ab),accuracy_score(y_test, y_pred_bg))


In [ ]:


Rf_Ab_Bg(dict_label_bloc, 650, 150,50)


In [ ]:

Y=[]
for i in range(10):
    Y.append(Rf_Ab_Bg(dict_label_bloc, 650, 150,40))
Z=np.array(Y)
print(np.shape(Z))


In [ ]:
print(Z)

print(Z[:,0])
print(Z[:,1])
print(Z[:,2])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming Z has three columns corresponding to Random Forest, AdaBoost, and Bagging
x = np.arange(0, 10, 1)

# Création de la figure
plt.figure()

# Tracé des courbes avec légendes
plt.plot(x, Z[:, 0], color='green', label='Random Forest')
plt.plot(x, Z[:, 1], color='orange', label='AdaBoost')
plt.plot(x, Z[:, 2], color='blue', label='Bagging')

# Ajout de la légende
plt.legend()

# Ajout des titres et étiquettes d'axe
plt.title("Comparison of AdaBoost, Bagging, and Random Forest over multiple classifications")
plt.xlabel('Classification')
plt.ylabel('Accuracy score')

# Affichage du graphique
plt.show()


Obersvation de la pertinence d'appliquer un Boosting ou un Bagging (ci dessus)

In [ ]:
def calculate_weights(X_test, y_test, classifiers):
    weights = {}
    for name, classifier in classifiers:
        y_pred_single = classifier.predict(X_test)
        accuracy_single = accuracy_score(y_test, y_pred_single)
        weights[name] = accuracy_single
    total_weight = sum(weights.values())
    return {name: weight / total_weight for name, weight in weights.items()}


In [ ]:
#VOTING WITH DIFFERENT CLASSIFICATIONS

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

def voting(dictionnaire, classifiers):
    data_flat = [element for sous_liste in dictionnaire.values() for element in sous_liste]
    data_flat = np.array(data_flat)
    data_flat2 = data_flat.reshape(320, -1)

    lab = np.array([[0] * 32, [1] * 32, [2] * 32, [3] * 32, [4] * 32, [5] * 32, [6] * 32, [7] * 32, [8] * 32, [9] * 32])
    lab = [element for sous_liste in lab for element in sous_liste]

    for i in range(len(lab)):
        lab[i] = np.array(lab[i])

    # Séparer les données en ensemble de test et d'apprentissage
    X_train, X_test, y_train, y_test = train_test_split(data_flat2, lab, test_size=0.25, random_state=7)

    # Create a VotingClassifier with various types of classifiers
    voting_classifier = VotingClassifier(estimators=classifiers, voting='soft')  # Use 'hard' for hard voting

    # Train each individual classifier before training the ensemble
    for name, classifier in classifiers:
        classifier.fit(X_train, y_train)

    # Train the ensemble classifier
    voting_classifier.fit(X_train, y_train)

    # Make predictions
    y_pred = voting_classifier.predict(X_test)

    # Evaluate accuracy for each classifier
    accuracies = {}
    for name, classifier in classifiers:
        y_pred_single = classifier.predict(X_test)
        accuracy_single = accuracy_score(y_test, y_pred_single)
        accuracies[name] = accuracy_single
        print(f'{name} Accuracy: {accuracy_single:.2f}')

    # Evaluate overall ensemble accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Ensemble Accuracy: {accuracy:.2f}')

    return accuracies, accuracy

# Example usage
classifiers = [
    ('rf', RandomForestClassifier(n_estimators=440, random_state=1)),
    ('ada', AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=440, random_state=1),n_estimators=10, random_state=2)),
    ('svc', SVC(probability=True, random_state=3)),
    ('dt', DecisionTreeClassifier(random_state=5)),
    ('lr', LogisticRegression(random_state=4)),
    ('knn', KNeighborsClassifier(n_neighbors=2)),
    ('nb', GaussianNB()),
    ('mlp', MLPClassifier(random_state=6)),
    ('bagging', BaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=440, random_state=1), n_estimators=40, random_state=7)),
    ('extra_trees', ExtraTreesClassifier(n_estimators=100, random_state=9)),
]

# Call the function with the classifiers
classifier_accuracies, ensemble_accuracy = voting(dict_label_bloc, classifiers)

# Access individual classifier accuracies
print("\nIndividual Classifier Accuracies:")
for name, accuracy in classifier_accuracies.items():
    print(f'{name}: {accuracy:.2f}')

# Access overall ensemble accuracy
print(f'\nOverall Ensemble Accuracy: {ensemble_accuracy:.2f}')


On eneleve les classifier donnant des scores inférieurs à 40%. Et on ajoute une notion de poids concernant les votes proportionnels à l'accuracy du classifieur

Let's try with only either Adaboost, rf or bagging because the classification is redundant

We have very similar results with way better speed

In [ ]:
#VOTING WITH DIFFERENT CLASSIFICATIONS

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

def voting(dictionnaire, classifiers):
    data_flat = [element for sous_liste in dictionnaire.values() for element in sous_liste]
    data_flat = np.array(data_flat)
    data_flat2 = data_flat.reshape(320, -1)

    lab = np.array([[0] * 32, [1] * 32, [2] * 32, [3] * 32, [4] * 32, [5] * 32, [6] * 32, [7] * 32, [8] * 32, [9] * 32])
    lab = [element for sous_liste in lab for element in sous_liste]

    for i in range(len(lab)):
        lab[i] = np.array(lab[i])

    # Séparer les données en ensemble de test et d'apprentissage
    X_train, X_test, y_train, y_test = train_test_split(data_flat2, lab, test_size=0.25, random_state=7)

    # Create a VotingClassifier with various types of classifiers
    voting_classifier = VotingClassifier(estimators=classifiers, voting='soft')  # Use 'hard' for hard voting

    # Train each individual classifier before training the ensemble
    for name, classifier in classifiers:
        classifier.fit(X_train, y_train)

    # Train the ensemble classifier
    voting_classifier.fit(X_train, y_train)

    # Make predictions
    y_pred = voting_classifier.predict(X_test)

    # Evaluate accuracy for each classifier
    accuracies = {}
    for name, classifier in classifiers:
        y_pred_single = classifier.predict(X_test)
        accuracy_single = accuracy_score(y_test, y_pred_single)
        accuracies[name] = accuracy_single
        print(f'{name} Accuracy: {accuracy_single:.2f}')

    # Evaluate overall ensemble accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Ensemble Accuracy: {accuracy:.2f}')
    for name, classifier in classifiers:
        classifier.fit(X_train, y_train)

    # Calculate weights based on accuracy
    classifier_weights = calculate_weights(X_test, y_test, classifiers)
    print(classifier_weights)
    return accuracies, accuracy
    voting_classifier = VotingClassifier(estimators=classifiers, voting='soft', weights=list(classifier_weights.values()))

# Example usage
classifiers = [
    
    ('ABrf1', AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=825, random_state=4), n_estimators=600, random_state=7)),
    ('ABrf2', AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=825, random_state=6), n_estimators=600, random_state=8)),
    ('ABrf3', AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=825, random_state=4), n_estimators=600, random_state=9)),
    ]

# Call the function with the classifiers
classifier_accuracies, ensemble_accuracy = voting(dict_label_bloc, classifiers)

# Access individual classifier accuracies
print("\nIndividual Classifier Accuracies:")
for name, accuracy in classifier_accuracies.items():
    print(f'{name}: {accuracy:.2f}')

# Access overall ensemble accuracy
print(f'\nOverall Ensemble Accuracy: {ensemble_accuracy:.2f}')



In [ ]:
    ('extra_trees1', ExtraTreesClassifier(n_estimators=550, random_state=1)),
    ('extra_trees2', ExtraTreesClassifier(n_estimators=550, random_state=3)),
    ('extra_trees3', ExtraTreesClassifier(n_estimators=550, random_state=5))


Attempt at RNN classification

In [ ]:
data_flat = [element for sous_liste in dict_label_bloc.values() for element in sous_liste]
data_flat = np.array(data_flat)

# Assuming original data shape is (320, 20, 8, 6)
num_samples, time_steps, x_dim, y_dim = data_flat.shape

# Flatten along x and y axes
data_flat2 = data_flat.reshape((num_samples, time_steps, -1))

# data_flat2 now has shape (320, 20, 8*6)

#shape transformation to fit an RNN


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical

# Assuming you have your data in X (features) and y (labels)

data_flat = [element for sous_liste in dict_label_bloc.values() for element in sous_liste]
data_flat = np.array(data_flat)
# Assuming original data shape is (320, 20, 8, 6)
num_samples, time_steps, x_dim, y_dim = data_flat.shape

# Flatten along x and y axes, keeping the time axis intact
data_flat2 = data_flat.reshape((num_samples, time_steps, -1))  

# Convert labels to one-hot encoding
y_one_hot = to_categorical(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_flat2, y_one_hot, test_size=0.25, random_state=42)

# Build a simple RNN model for categorical classification
rnn_model = models.Sequential([
    layers.SimpleRNN(64, activation='relu', input_shape=(time_steps, x_dim * y_dim)),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 classes for categorical classification
])

rnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the RNN model
rnn_model.fit(X_train, y_train, epochs=25, batch_size=32, validation_split=0.2)

# Evaluate RNN model on test set
y_pred_rnn = rnn_model.predict(X_test)
y_pred_rnn_classes = tf.argmax(y_pred_rnn, axis=1)
y_test_classes = tf.argmax(y_test, axis=1)

accuracy_rnn = accuracy_score(y_test_classes, y_pred_rnn_classes)
print(f'RNN Accuracy: {accuracy_rnn:.2f}')


In [ ]:

#RANDOM FOREST BOOSTED WITH TREE ON VOTING CLASSFIER FOR TEST DATA WITH ALL TRAINING DATA

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier



# Chargez les données du dictionnaire
data_flat = [element for sous_liste in dict_label_bloc.values() for element in sous_liste]
data_flat=np.array(data_flat)
data_flat2=data_flat.reshape(320,-1)

lab=np.array([[0]*32,[1]*32,[2]*32,[3]*32,[4]*32,[5]*32,[6]*32,[7]*32,[8]*32,[9]*32])
lab = [element for sous_liste in lab for element in sous_liste]

for i in range(len(lab)):
    lab[i]=np.array(lab[i])
# Séparez les données en ensemble de test et d'apprentissage

X_train2 = data_flat2
y_train2 = lab
#les data pour la classification du voting

# Séparer les données en ensemble de test et d'apprentissage
X_train, X_test, y_train, y_test = train_test_split(data_flat2, lab, test_size=0.25, random_state=7)
#les data pour les classifications dans le boosting pour avoir une accuracy

#Classifiers :    
classifiers = []

# Assuming you want 10 sets of AdaBoost with RandomForest and 10 sets of ExtraTrees classifiers
num_classifiers = 5

for i in range(1, num_classifiers + 1):
    ab_rf_classifier = AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=825, random_state=i), n_estimators=300, random_state=i + 10)
    extra_trees_classifier = ExtraTreesClassifier(n_estimators=800, random_state=i + 20)
    classifiers.append(('AB_rf_{}'.format(i), ab_rf_classifier))
    classifiers.append(('extra_trees_{}'.format(i), extra_trees_classifier))
    
#Creation of voting classifier :
    
# Create a VotingClassifier with various types of classifiers
voting_classifier = VotingClassifier(estimators=classifiers, voting='soft')  # Use 'hard' for hard voting

# Train each individual classifier before training the ensemble
for name, classifier in classifiers:
    classifier.fit(X_train, y_train)

# Train the ensemble classifier
voting_classifier.fit(X_train, y_train)

# Make predictions
y_pred = voting_classifier.predict(X_test)

# Evaluate accuracy for each classifier
accuracies = {}
for name, classifier in classifiers:
    y_pred_single = classifier.predict(X_test)
    accuracy_single = accuracy_score(y_test, y_pred_single)
    accuracies[name] = accuracy_single
    print(f'{name} Accuracy: {accuracy_single:.2f}')

# Evaluate overall ensemble accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Ensemble Accuracy: {accuracy:.2f}')
for name, classifier in classifiers:
    classifier.fit(X_train, y_train)

# Calculate weights based on accuracy
classifier_weights = calculate_weights(X_test, y_test, classifiers)
voting_classifier = VotingClassifier(estimators=classifiers, voting='soft', weights=list(classifier_weights.values()))

# Entraînez le modèle sur l'ensemble d'apprentissage
voting_classifier.fit(X_train2, y_train2)

In [ ]:
#VOTING WITH VARIOUS CLASSIFIERS

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier


# Assuming you have your data in X (features) and y (labels)





# Chargez les données du dictionnaire
data_flat = [element for sous_liste in dict_label_bloc.values() for element in sous_liste]
data_flat=np.array(data_flat)
data_flat2=data_flat.reshape(320,-1)

lab=np.array([[0]*32,[1]*32,[2]*32,[3]*32,[4]*32,[5]*32,[6]*32,[7]*32,[8]*32,[9]*32])
lab = [element for sous_liste in lab for element in sous_liste]

for i in range(len(lab)):
    lab[i]=np.array(lab[i])
# Séparez les données en ensemble de test et d'apprentissage

X_train2 = data_flat2
y_train2 = lab
#les data pour la classification du voting

# Séparer les données en ensemble de test et d'apprentissage
X_train, X_test, y_train, y_test = train_test_split(data_flat2, lab, test_size=0.25, random_state=7)
#les data pour les classifications dans le boosting pour avoir une accuracy

#Classifiers :    
classifiers = [
    ('rf', RandomForestClassifier(n_estimators=440, random_state=1)),
    ('dt', DecisionTreeClassifier(random_state=5)),
    ('lr', LogisticRegression(random_state=4)),
    ('knn', KNeighborsClassifier(n_neighbors=2)),
    ('extra_trees', ExtraTreesClassifier(n_estimators=100, random_state=9)),
]
#Creation of voting classifier :
    
# Create a VotingClassifier with various types of classifiers
voting_classifier = VotingClassifier(estimators=classifiers, voting='soft')  # Use 'hard' for hard voting

# Train each individual classifier before training the ensemble
for name, classifier in classifiers:
    classifier.fit(X_train, y_train)

# Train the ensemble classifier
voting_classifier.fit(X_train, y_train)

# Make predictions
y_pred = voting_classifier.predict(X_test)

# Evaluate accuracy for each classifier
accuracies = {}
for name, classifier in classifiers:
    y_pred_single = classifier.predict(X_test)
    accuracy_single = accuracy_score(y_test, y_pred_single)
    accuracies[name] = accuracy_single
    print(f'{name} Accuracy: {accuracy_single:.2f}')

# Evaluate overall ensemble accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Ensemble Accuracy: {accuracy:.2f}')
for name, classifier in classifiers:
    classifier.fit(X_train, y_train)

# Calculate weights based on accuracy
classifier_weights = calculate_weights(X_test, y_test, classifiers)
voting_classifier3 = VotingClassifier(estimators=classifiers, voting='soft', weights=list(classifier_weights.values()))

# Entraînez le modèle sur l'ensemble d'apprentissage
voting_classifier3.fit(X_train2, y_train2)



In [ ]:
y_pred_voting3=voting_classifier3.predict(data_flat2_test)

import pandas as pd
import numpy as np
# Remplacez cela par vos prédictions réelles
df = pd.DataFrame(y_pred_voting3, columns=['label'])

# Mapping des remplacements
mapping = {
    0: 'Ecole',
    1: 'Joyeux',
    2: 'Decider',
    3: 'Carnaval',
    4: 'Pyjama',
    5: 'Ruisseau',
    6: 'Addition',
    7: 'Huitre',
    8: 'Fillette',
    9: 'Musique'
}

# Remplacement des valeurs dans le DataFrame
df.replace({'label': mapping}, inplace=True)

# Ajouter une colonne 'id' contenant l'indice de ligne
df.insert(0, 'id', df.index)

# Sauvegarder le DataFrame en tant que fichier CSV avec une première ligne "id,label"
csv_filename = "submission.csv"
df.to_csv(csv_filename, index=False)

# Lecture du fichier CSV pour vérification
test_output = pd.read_csv(csv_filename)

# Affichage du DataFrame
print(df)

New attempt with non boosted random forests (3rd)

In [ ]:
#RANDOM FOREST WITH XtremeTREE ON VOTING CLASSFIER FOR TEST DATA WITH ALL TRAINING DATA

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier



# Chargez les données du dictionnaire
data_flat = [element for sous_liste in dict_label_bloc.values() for element in sous_liste]
data_flat=np.array(data_flat)
data_flat2=data_flat.reshape(320,-1)

lab=np.array([[0]*32,[1]*32,[2]*32,[3]*32,[4]*32,[5]*32,[6]*32,[7]*32,[8]*32,[9]*32])
lab = [element for sous_liste in lab for element in sous_liste]

for i in range(len(lab)):
    lab[i]=np.array(lab[i])
# Séparez les données en ensemble de test et d'apprentissage

X_train2 = data_flat2
y_train2 = lab
#les data pour la classification du voting

# Séparer les données en ensemble de test et d'apprentissage
X_train, X_test, y_train, y_test = train_test_split(data_flat2, lab, test_size=0.25, random_state=7)
#les data pour les classifications dans le boosting pour avoir une accuracy

#Classifiers :    
classifiers = []

# Assuming you want 10 sets of AdaBoost with RandomForest and 10 sets of ExtraTrees classifiers
num_classifiers = 5

for i in range(1, num_classifiers + 1):
    rf_classifier = RandomForestClassifier(n_estimators=825, random_state=i)
    extra_trees_classifier = ExtraTreesClassifier(n_estimators=800, random_state=i + 20)
    classifiers.append(('rf_{}'.format(i), rf_classifier))
    classifiers.append(('extra_trees_{}'.format(i), extra_trees_classifier))
    
#Creation of voting classifier :
    
# Create a VotingClassifier with various types of classifiers
voting_classifier = VotingClassifier(estimators=classifiers, voting='soft')  # Use 'hard' for hard voting

# Train each individual classifier before training the ensemble
for name, classifier in classifiers:
    classifier.fit(X_train, y_train)

# Train the ensemble classifier
voting_classifier.fit(X_train, y_train)

# Make predictions
y_pred = voting_classifier.predict(X_test)

# Evaluate accuracy for each classifier
accuracies = {}
for name, classifier in classifiers:
    y_pred_single = classifier.predict(X_test)
    accuracy_single = accuracy_score(y_test, y_pred_single)
    accuracies[name] = accuracy_single
    print(f'{name} Accuracy: {accuracy_single:.2f}')

# Evaluate overall ensemble accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Ensemble Accuracy: {accuracy:.2f}')
for name, classifier in classifiers:
    classifier.fit(X_train, y_train)

# Calculate weights based on accuracy
classifier_weights = calculate_weights(X_test, y_test, classifiers)
voting_classifier2 = VotingClassifier(estimators=classifiers, voting='soft', weights=list(classifier_weights.values()))

# Entraînez le modèle sur l'ensemble d'apprentissage
voting_classifier2.fit(X_train2, y_train2)

Production of the third submission

In [ ]:
y_pred_voting2=voting_classifier2.predict(data_flat2_test)

import pandas as pd
import numpy as np
# Remplacez cela par vos prédictions réelles
df = pd.DataFrame(y_pred_voting2, columns=['label'])

# Mapping des remplacements
mapping = {
    0: 'Ecole',
    1: 'Joyeux',
    2: 'Decider',
    3: 'Carnaval',
    4: 'Pyjama',
    5: 'Ruisseau',
    6: 'Addition',
    7: 'Huitre',
    8: 'Fillette',
    9: 'Musique'
}

# Remplacement des valeurs dans le DataFrame
df.replace({'label': mapping}, inplace=True)

# Ajouter une colonne 'id' contenant l'indice de ligne
df.insert(0, 'id', df.index)

# Sauvegarder le DataFrame en tant que fichier CSV avec une première ligne "id,label"
csv_filename = "submission.csv"
df.to_csv(csv_filename, index=False)

# Lecture du fichier CSV pour vérification
test_output = pd.read_csv(csv_filename)

# Affichage du DataFrame
print(df)

In [ ]:
y_pred_voting2=voting_classifier2.predict(data_flat2_test)

import pandas as pd
import numpy as np
# Remplacez cela par vos prédictions réelles
df = pd.DataFrame(y_pred_voting2, columns=['label'])

# Mapping des remplacements
mapping = {
    0: 'Ecole',
    1: 'Joyeux',
    2: 'Decider',
    3: 'Carnaval',
    4: 'Pyjama',
    5: 'Ruisseau',
    6: 'Addition',
    7: 'Huitre',
    8: 'Fillette',
    9: 'Musique'
}

# Remplacement des valeurs dans le DataFrame
df.replace({'label': mapping}, inplace=True)

# Ajouter une colonne 'id' contenant l'indice de ligne
df.insert(0, 'id', df.index)

# Sauvegarder le DataFrame en tant que fichier CSV avec une première ligne "id,label"
csv_filename = "submission.csv"
df.to_csv(csv_filename, index=False)

# Lecture du fichier CSV pour vérification
test_output = pd.read_csv(csv_filename)

# Affichage du DataFrame
print(df)

In [ ]:
#RANDOM FOREST CLASSFIER FOR TEST DATA WITH ALL TRAINING DATA

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier



# Chargez les données du dictionnaire
data_flat = [element for sous_liste in dict_label_bloc.values() for element in sous_liste]
data_flat=np.array(data_flat)
data_flat2=data_flat.reshape(320,-1)

lab=np.array([[0]*32,[1]*32,[2]*32,[3]*32,[4]*32,[5]*32,[6]*32,[7]*32,[8]*32,[9]*32])
lab = [element for sous_liste in lab for element in sous_liste]

for i in range(len(lab)):
    lab[i]=np.array(lab[i])
# Séparez les données en ensemble de test et d'apprentissage

X_train = data_flat2
y_train = lab

# Créez un modèle Random Forest fair varier le n__estimator
clf = RandomForestClassifier(n_estimators=650)

# Entraînez le modèle sur l'ensemble d'apprentissage
clf.fit(X_train, y_train)



In [ ]:
#BOOSTED RANDOM FOREST CLASSFIER FOR TEST DATA WITH ALL TRAINING DATA

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


# Chargez les données du dictionnaire
data_flat = [element for sous_liste in dict_label_bloc.values() for element in sous_liste]
data_flat=np.array(data_flat)
data_flat2=data_flat.reshape(320,-1)

lab=np.array([[0]*32,[1]*32,[2]*32,[3]*32,[4]*32,[5]*32,[6]*32,[7]*32,[8]*32,[9]*32])
lab = [element for sous_liste in lab for element in sous_liste]

for i in range(len(lab)):
    lab[i]=np.array(lab[i])
# Séparez les données en ensemble de test et d'apprentissage

X_train = data_flat2
y_train = lab

# Créez un modèle Random Forest fair varier le n__estimator
AB_rf = AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators=150, random_state=4), n_estimators=10, random_state= 12)

# Entraînez le modèle sur l'ensemble d'apprentissage
AB_rf.fit(X_train, y_train)



prediction 5

In [ ]:
y_pred5 = AB_rf.predict(data_flat2_test)

Creation of the prediction 1 

In [ ]:
y_pred = clf.predict(data_flat2_test)

creation of the prediction 2

In [ ]:
y_pred_voting = voting_classifier.predict(data_flat2_test)

Exportation of the first attempts

In [ ]:
import pandas as pd
import numpy as np
# Remplacez cela par vos prédictions réelles
df = pd.DataFrame(y_pred, columns=['label'])

# Mapping des remplacements
mapping = {
    0: 'Ecole',
    1: 'Joyeux',
    2: 'Decider',
    3: 'Carnaval',
    4: 'Pyjama',
    5: 'Ruisseau',
    6: 'Addition',
    7: 'Huitre',
    8: 'Fillette',
    9: 'Musique'
}

# Remplacement des valeurs dans le DataFrame
df.replace({'label': mapping}, inplace=True)

# Ajouter une colonne 'id' contenant l'indice de ligne
df.insert(0, 'id', df.index)

# Sauvegarder le DataFrame en tant que fichier CSV avec une première ligne "id,label"
csv_filename = "submission.csv"
df.to_csv(csv_filename, index=False)

# Lecture du fichier CSV pour vérification
test_output = pd.read_csv(csv_filename)

# Affichage du DataFrame
print(df)

production du csv

In [ ]:
import pandas as pd
import numpy as np
# Remplacez cela par vos prédictions réelles
df = pd.DataFrame(y_pred5, columns=['label'])

# Mapping des remplacements
mapping = {
    0: 'Ecole',
    1: 'Joyeux',
    2: 'Decider',
    3: 'Carnaval',
    4: 'Pyjama',
    5: 'Ruisseau',
    6: 'Addition',
    7: 'Huitre',
    8: 'Fillette',
    9: 'Musique'
}

# Remplacement des valeurs dans le DataFrame
df.replace({'label': mapping}, inplace=True)

# Ajouter une colonne 'id' contenant l'indice de ligne
df.insert(0, 'id', df.index)

# Sauvegarder le DataFrame en tant que fichier CSV avec une première ligne "id,label"
csv_filename = "submission.csv"
df.to_csv(csv_filename, index=False)

# Lecture du fichier CSV pour vérification
test_output = pd.read_csv(csv_filename)

# Affichage du DataFrame
print(df)

Exportation of the second attempt

In [ ]:
import pandas as pd
import numpy as np
# Remplacez cela par vos prédictions réelles
df = pd.DataFrame(y_pred_voting, columns=['label'])

# Mapping des remplacements
mapping = {
    0: 'Ecole',
    1: 'Joyeux',
    2: 'Decider',
    3: 'Carnaval',
    4: 'Pyjama',
    5: 'Ruisseau',
    6: 'Addition',
    7: 'Huitre',
    8: 'Fillette',
    9: 'Musique'
}

# Remplacement des valeurs dans le DataFrame
df.replace({'label': mapping}, inplace=True)

# Ajouter une colonne 'id' contenant l'indice de ligne
df.insert(0, 'id', df.index)

# Sauvegarder le DataFrame en tant que fichier CSV avec une première ligne "id,label"
csv_filename = "submission.csv"
df.to_csv(csv_filename, index=False)

# Lecture du fichier CSV pour vérification
test_output = pd.read_csv(csv_filename)

# Affichage du DataFrame
print(df)